## ДЗ 5:
### Экспорт моделей в различные форматы OpenVino, ONNX

In [2]:
import ultralytics
from ultralytics import YOLO

In [3]:
import torch
import time
import torch.quantization
from torch import profiler
from torch.profiler import profile, record_function, ProfilerActivity

In [4]:
from utils import get_model_size, profile_model

In [5]:
torch.seed = 42

### Исходная модель

In [6]:
yolo = YOLO('yolov8s.pt')
model = yolo.model
model.fuse()
model.eval()
print()

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


In [7]:
print(profile_model(model))
print(f'model size: {get_model_size(model)}')

STAGE:2023-10-16 19:06:10 20073:2172288 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-10-16 19:06:10 20073:2172288 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-16 19:06:10 20073:2172288 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total KFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        -1.13%   -1315.000us       100.00%     116.299ms     116.299ms           0 b     -24.10 Mb             1            --  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 116.299ms

model size: 42.558837890625


### ONNX

In [8]:
yolo.export(format='onnx')

Ultralytics YOLOv8.0.190 🚀 Python-3.8.0 torch-2.0.1 CPU (Apple M1 Pro)

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 1.7s, saved as 'yolov8s.onnx' (42.8 MB)

Export complete (5.6s)
Results saved to /Users/timur/Documents/itmo_master/1_term/model_compression/IonovTimur
Predict:         yolo predict task=detect model=yolov8s.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8s.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'yolov8s.onnx'

In [9]:
input_example = torch.randn(1, 3, 640, 640)

In [10]:
# pip insall onnxruntime

In [11]:
import onnxruntime as ort
import numpy as np

In [12]:
ort_sess = ort.InferenceSession('yolov8s.onnx')
outputs = ort_sess.run(None, {'images': input_example.float().numpy()})

In [13]:
import time

In [14]:
s = time.time()
ort_sess.run(None, {'images': input_example.float().numpy()})
print(time.time() - s)

0.18231797218322754


In [15]:
s = time.time()
yolo.model(input_example)
print(time.time() - s)

0.7649898529052734


## OpenVino

In [16]:
yolo.export(format='openvino')

Ultralytics YOLOv8.0.190 🚀 Python-3.8.0 torch-2.0.1 CPU (Apple M1 Pro)

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 1.6s, saved as 'yolov8s.onnx' (42.8 MB)
requirements: Ultralytics requirement ['openvino-dev>=2023.0'] not found, attempting AutoUpdate...


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================




[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for openvino-dev>=2023.0 from https://files.pythonhosted.org/packages/86/65/76ce426ac1e1d7db560187440b307be10af108a92d28e3e1842e388989db/openvino_dev-2023.1.0-12185-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for openvino-telemetry>=2022.1.0 from https://files.pythonhosted.org/packages/18/7e/656e52224ef145abed0c92535c786181b933f87323fc88824e947f611119/openvino_telemetry-2023.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for texttable>=1.6.3 from https://files.pythonhosted.org/packages/24/99/4772b8e00a136f3e01236de33b0efda31ee7077203ba5967fcc76da94d65/texttable-1.7.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for openvino==2023.1.0 from https://files.pythonhosted.org/packages/30/dd/eadf

'yolov8s_openvino_model'

In [17]:
from openvino.inference_engine import IECore

/var/folders/jq/snmzydvn60x9yz78t64j0__m0000gn/T/ipykernel_20073/2158470354.py:1: FutureWarning: OpenVINO Inference Engine Python API is deprecated and will be removed in 2024.0 release.For instructions on transitioning to the new API, please refer to https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
  from openvino.inference_engine import IECore


In [18]:
ie = IECore()
net = ie.read_network(model="yolov8s_openvino_model/yolov8s.xml", weights="yolov8s_openvino_model/yolov8s.bin")
exec_net = ie.load_network(network=net, device_name="CPU")

input_blob = next(iter(net.input_info))
out_blob = next(iter(net.outputs))

In [19]:
s = time.time()
exec_net.infer(inputs={input_blob: input_example})
print(time.time() - s)

0.28099513053894043


In [ ]:
{DET_MODEL_NAME}_openvino_model/{DET_MODEL_NAME}.xml"
if not det_model_path.exists():
    det_model.export(format="openvino", dynamic=True, half=False)